In [1]:
# I'll try to downgrade pandas since the new version doesnt have .appand funcion
!pip install pandas==1.5.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.0 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.1 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2024.10.0 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import scipy.stats as sp
from tqdm import tqdm

In [3]:
def ofc_p_value (f = 50, h = 14, r_g = 7, n_g = 1, r_b = 35, n_b = 1, d = 100, v = 45, r_s = 60, a = [0]):
    c = 0.009
    ofc1 =(f*c*h*60)/((r_g/n_g)+(r_b/n_b)+(60*d/v)+(r_s))
    ofc2 =(24*60*c)/((r_g/n_g)+(r_b/n_b))
    ofc = min(ofc1, ofc2)
    t, p = sp.ttest_1samp(a, ofc)
    return (ofc, p)

In [4]:
df_gate = pd.read_csv("2024-11-13_Gate_Num_Exp1_30iter_Results.csv", sep = ',')
df_gate['Flow_MMgal'] = df_gate['Flow']/(10**6)
r_g = df_gate['Gate_Time'].unique()

In [5]:
df_bay = pd.read_csv('2024-11-13_Bay_Num_Exp1_30iter_Results.csv', sep = ',')
df_bay['Flow_MMgal'] = df_bay['Flow']/(10**6)
r_b = df_bay['Bay_Time'].unique()

In [6]:
df_speed = pd.read_csv('2024-11-13_Speed_Num_Exp_30iter_Results.csv', sep = ',')
df_speed['Flow_MMgal'] = df_speed['Flow']/(10**6)
v = df_speed['Truck_Speed'].unique()

In [7]:
df_hos = pd.read_csv('2024-11-13_HoS_Num_Exp_30iter_Results.csv', sep = ',')
df_hos = df_hos[~(df_hos['Truck_HoS'] == 24)]
df_hos['Flow_MMgal'] = df_hos['Flow']/(10**6)
h = df_hos['Truck_HoS'].unique()

In [8]:
df_fleet = pd.read_csv('2024-11-13_Fleet_Num_Exp_30iter_Results.csv', sep = ',')
df_fleet['Flow_MMgal'] = df_fleet['Flow']/(10**6)
f = df_fleet['Fleet'].unique()

In [9]:
n_g = df_gate['Gates'].unique()
n_b = df_gate['Bays'].unique()
d = df_gate['Station_Dist'].unique()

In [10]:

df_results1 = pd.DataFrame(columns = ['Gates', 'Bays', 'Station_Dist', 'Parameter', 'Value', 'p-value', 'DES<ANA'])

In [11]:
for i in tqdm(range(len(n_g))):
    for j in tqdm(range(len(n_b))):
        for k in range(len(d)):
            df_dum_gate = df_gate[(df_gate['Gates'] == n_g[i]) & (df_gate['Bays'] == n_b[j]) &\
                                  (df_gate['Station_Dist'] == d[k])]
            for l in range(len(r_g)):
                des_values = df_dum_gate[df_dum_gate['Gate_Time'] == r_g[l]]['Flow_MMgal'].values
                ana, p_value = ofc_p_value(r_g = r_g[l], n_g = n_g[i], n_b = n_b[j], d = d[k], a = des_values)
                x = np.mean(des_values) <= ana
                df_results1 = df_results1.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
                                                  'Parameter': 'Gate', 'Value': r_g[l], 'p-value': p_value,\
                                                  'DES<ANA': x}, ignore_index = True)

            df_dum_bay = df_bay[(df_bay['Gates'] == n_g[i]) & (df_bay['Bays'] == n_b[j]) &\
                                (df_bay['Station_Dist'] == d[k])]
            for l in range(len(r_b)):
                des_values = df_dum_bay[df_dum_bay['Bay_Time'] == r_b[l]]['Flow_MMgal'].values
                ana, p_value = ofc_p_value(r_b = r_b[l], n_g = n_g[i], n_b = n_b[j], d = d[k], a = des_values)
                x = np.mean(des_values) <= ana
                df_results1 = df_results1.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
                                                  'Parameter': 'Bay', 'Value': r_b[l], 'p-value': p_value,\
                                                  'DES<ANA': x}, ignore_index = True)

            df_dum_speed = df_speed[(df_speed['Gates'] == n_g[i]) & (df_speed['Bays'] == n_b[j]) &\
                                    (df_speed['Station_Dist'] == d[k])]
            for l in range(len(v)):
                des_values = df_dum_speed[df_dum_speed['Truck_Speed'] == v[l]]['Flow_MMgal'].values
                ana, p_value = ofc_p_value(v = v[l], n_g = n_g[i], n_b = n_b[j], d = d[k], a = des_values)
                x = np.mean(des_values) <= ana
                df_results1 = df_results1.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
                                                  'Parameter': 'Speed', 'Value': v[l], 'p-value': p_value,\
                                                  'DES<ANA': x}, ignore_index = True)

            df_dum_hos = df_hos[(df_hos['Gates'] == n_g[i]) & (df_hos['Bays'] == n_b[j]) &\
                                (df_hos['Station_Dist'] == d[k])]
            for l in range(len(h)):
                des_values = df_dum_hos[df_dum_hos['Truck_HoS'] == h[l]]['Flow_MMgal'].values
                ana, p_value = ofc_p_value(h = h[l], n_g = n_g[i], n_b = n_b[j], d = d[k], a = des_values)
                x = np.mean(des_values) <= ana
                df_results1 = df_results1.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
                                                  'Parameter': 'HoS', 'Value': h[l], 'p-value': p_value,\
                                                  'DES<ANA': x}, ignore_index = True)

            df_dum_fleet = df_fleet[(df_fleet['Gates'] == n_g[i]) & (df_fleet['Bays'] == n_b[j]) &\
                                    (df_fleet['Station_Dist'] == d[k])]
            for l in range(len(f)):
                des_values = df_dum_fleet[df_dum_fleet['Fleet'] == f[l]]['Flow_MMgal'].values
                ana, p_value = ofc_p_value(f = f[l], n_g = n_g[i], n_b = n_b[j], d = d[k], a = des_values)
                x = np.mean(des_values) <= ana
                df_results1 = df_results1.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
                                                  'Parameter': 'Fleet', 'Value': f[l], 'p-value': p_value,\
                                                  'DES<ANA': x}, ignore_index = True)

Streaming output truncated to the last 5000 lines.
<ipython-input-11-199367cbb947>:30: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_results1 = df_results1.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
<ipython-input-11-199367cbb947>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results1 = df_results1.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
<ipython-input-11-199367cbb947>:30: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_results1 = df_results1.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
<ipython-input-11-199367cbb947>:30: FutureWarning: The frame.append method is deprecated 

In [13]:
df_results1.to_csv('2024-11-17_Num_Exp_DES_v_ANA_ttest.csv')

In [14]:
df_results2 = pd.DataFrame(columns = ['Gates', 'Bays', 'Station_Dist', 'Parameter', 'Value', 'p-value', 'DES_Mono',\
                                      'Avg_OFC'])

In [15]:
for i in tqdm(range(len(n_g))):
    for j in tqdm(range(len(n_b))):
        for k in range(len(d)):
            df_dum_gate = df_gate[(df_gate['Gates'] == n_g[i]) & (df_gate['Bays'] == n_b[j]) &\
                                  (df_gate['Station_Dist'] == d[k])]
            for l in range(len(r_g)-1):
                a = df_dum_gate[df_dum_gate['Gate_Time'] == r_g[l]]['Flow_MMgal'].values
                b = df_dum_gate[df_dum_gate['Gate_Time'] == r_g[l+1]]['Flow_MMgal'].values
                t, p_value = sp.ttest_ind(a, b, equal_var = False)
                x = np.mean(a) >= np.mean(b)
                df_results2 = df_results2.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
                                                  'Parameter': 'Gate', 'Value': r_g[l], 'p-value': p_value,\
                                                  'DES_Mono': x, 'Avg_OFC': np.mean(a)}, ignore_index = True)

            df_dum_bay = df_bay[(df_bay['Gates'] == n_g[i]) & (df_bay['Bays'] == n_b[j]) &\
                                (df_bay['Station_Dist'] == d[k])]
            for l in range(len(r_b)-1):
                a = df_dum_bay[df_dum_bay['Bay_Time'] == r_b[l]]['Flow_MMgal'].values
                b = df_dum_bay[df_dum_bay['Bay_Time'] == r_b[l+1]]['Flow_MMgal'].values
                t, p_value = sp.ttest_ind(a, b, equal_var = False)
                x = np.mean(a) >= np.mean(b)
                df_results2 = df_results2.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
                                                  'Parameter': 'Bay', 'Value': r_b[l], 'p-value': p_value,\
                                                  'DES_Mono': x, 'Avg_OFC': np.mean(a)}, ignore_index = True)

            df_dum_speed = df_speed[(df_speed['Gates'] == n_g[i]) & (df_speed['Bays'] == n_b[j]) &\
                                    (df_speed['Station_Dist'] == d[k])]
            for l in range(len(v)-1):
                a = df_dum_speed[df_dum_speed['Truck_Speed'] == v[l]]['Flow_MMgal'].values
                b = df_dum_speed[df_dum_speed['Truck_Speed'] == v[l+1]]['Flow_MMgal'].values
                t, p_value = sp.ttest_ind(a, b, equal_var = False)
                x = np.mean(a) <= np.mean(b)
                df_results2 = df_results2.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
                                                  'Parameter': 'Speed', 'Value': v[l+1], 'p-value': p_value,\
                                                  'DES_Mono': x, 'Avg_OFC': np.mean(b)}, ignore_index = True)

            df_dum_hos = df_hos[(df_hos['Gates'] == n_g[i]) & (df_hos['Bays'] == n_b[j]) &\
                                (df_hos['Station_Dist'] == d[k])]
            for l in range(len(h)-1):
                a = df_dum_hos[df_dum_hos['Truck_HoS'] == h[l]]['Flow_MMgal'].values
                b = df_dum_hos[df_dum_hos['Truck_HoS'] == h[l+1]]['Flow_MMgal'].values
                t, p_value = sp.ttest_ind(a, b, equal_var = False)
                x = np.mean(a) <= np.mean(b)
                df_results2 = df_results2.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
                                                  'Parameter': 'HoS', 'Value': h[l+1], 'p-value': p_value,\
                                                  'DES_Mono': x, 'Avg_OFC': np.mean(b)}, ignore_index = True)

            df_dum_fleet = df_fleet[(df_fleet['Gates'] == n_g[i]) & (df_fleet['Bays'] == n_b[j]) &\
                                    (df_fleet['Station_Dist'] == d[k])]
            for l in range(len(f)-1):
                a = df_dum_fleet[df_dum_fleet['Fleet'] == f[l]]['Flow_MMgal'].values
                b = df_dum_fleet[df_dum_fleet['Fleet'] == f[l+1]]['Flow_MMgal'].values
                t, p_value = sp.ttest_ind(a, b, equal_var = False)
                x = np.mean(a) <= np.mean(b)
                df_results2 = df_results2.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
                                                  'Parameter': 'Fleet', 'Value': f[l+1], 'p-value': p_value,\
                                                  'DES_Mono': x, 'Avg_OFC': np.mean(b)}, ignore_index = True)

Streaming output truncated to the last 5000 lines.
  df_results2 = df_results2.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
<ipython-input-15-12d83156fbd4>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results2 = df_results2.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
<ipython-input-15-12d83156fbd4>:11: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_results2 = df_results2.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
<ipython-input-15-12d83156fbd4>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results2 = df_results2.append({'Gates': n_g[i], 'Bays': n_b[j],'Station_Dist': d[k],\
<ipython-input-15-12d83156fbd4>:22: Fu

In [16]:
df_results2.to_csv('2024-11-17_Num_Exp_DES_ttest.csv')